# Setup

In [0]:
!pip install magentic
!pip install openbb

In [0]:
import os
os.environ["OPENAI_API_KEY"] = "..."

In [0]:
from openbb import obb
obb.user.preferences.output_type="llm"
obb.system.python_settings.docstring_sections=["description", "parameters"]
obb.system.python_settings.docstring_max_length=1024

In [0]:
import openbb
openbb.build()

In [0]:
obb.account.login(pat="...")

# OpenBB Platform

In [0]:
from openbb import obb

In [0]:
obb.equity.profile?

# Example

In [0]:
from magentic import chatprompt, SystemMessage, UserMessage, FunctionCall

@chatprompt(
    SystemMessage("You are an expert financial analyst. Use the tools provided when answering a question."),
    UserMessage("{query}"),
    functions=[
        obb.equity.compare.peers,
        obb.equity.profile,
        obb.equity.estimates.price_target,
    ],
)
def ask(query: str) -> str | FunctionCall: ...

def execute(query: str) -> str:
    response = ask(query)
    fc_results = []

    while isinstance(response, FunctionCall):
        result = response()
        fc_results.append(f"Function `{response.function.__name__}` returned: {result}")
        response = ask(query=query + "\n\n" + "\n".join(fc_results))

    return response

In [0]:
query = """
Check what are TSLA peers. From those, check which one has the highest market cap.
Then, on the ticker that has the highest market cap get the most recent rating from an analyst.
And tell me who was the analyst and what date was it that the rating was done.
"""

answer = execute(query)
answer

In [0]:
peers = obb.equity.compare.peers("TSLA").to_df()
peers_list = list(peers["peers_list"])
peers_list

In [0]:
for peer in peers_list:
    print(peer)
    print(int(obb.equity.profile(peer).to_df()["market_cap"][0]))

In [0]:
obb.equity.estimates.price_target("GM").to_df()